# Challenge de detección de latidos

Se deberán clasificar unos latidos de test en las siguientes categorías:

N		Normal beat (displayed as "·" by the PhysioBank ATM, LightWAVE, pschart, and psfd)  
R		Right bundle branch block beat  
A		Atrial premature beat  
V		Premature ventricular contraction  

Para lo cual se cuenta con los siguientes datos de entrenamiento:

signals_train.pkl: una lista que en cada elemento contiene 12 derivaciones de un mismo latido. Es decir, para un latido en particular, se tienen 12 mediciones en puntos distintos del cuerpo.

annotations_train.pkl: una lista que contiene,para cada elemento de la lista anterior, la clasificación en {N, R, A, V} de el latido correspondiente.

patients_train.pkl: una lista que contiene, para cada elemento de la lista de señales, un string que identifica al paciente al que se le midió ese latido.

Para testing se brinda la siguiente información:

signals_test.pkl: idem signals_train.pkl

patients_test.pkl: idem patients_train.pkl

Aclaración:
Debido a problemas en el dataset original, fueron removidos los pacientes 4, 17, 35, 44, 57, 72 y 74, ya que sus anotaciones contenían errores. No se preocupen cuando no los vean.

Se deberá subir un archivo .csv que contenga dos columnas:

id, type

La columna id deberá contener el número de ítem de signals_test.pkl que se clasificó
La columna type deberá tener la clasificación que se le dió al latido nro id de la lista en signals_test.pkl

Se adjunta como ejemplo un archivo sample.csv con el formato que se deberá subir para ser evaluado.
Se evaluará el accuracy de clasificación de los latidos entregados en el subset de test. Dichos latidos son 20186.

In [ ]:
import os
# List data files that are connected to the kernel
os.listdir('../input/latidositbadatasset')

In [ ]:
os.listdir('../input/lab-ml-itba-clasificacion-de-latidos')

# Analizo el train

In [ ]:
# Cargo los datos de train
import pickle
with open("../input/lab-ml-itba-clasificacion-de-latidos/signals_train.pkl", 'rb') as f:
        lat=pickle.load(f)
with open("../input/latidositbadatasset/annotations_train.pkl", 'rb') as f:
        ann=pickle.load(f)
with open("../input/latidositbadatasset/patients_train.pkl", 'rb') as f:
        pat=pickle.load(f)

In [ ]:
# Grafico el latido nro lat_idx
import numpy as np
from random import randint
import matplotlib.pyplot as plot
lat_idx=randint(0,len(lat))
f, axarr = plot.subplots(12, sharex=True)
f.set_figheight(15)
f.set_figwidth(15)
print("Latido nro: {}".format(lat_idx))
print("El latido es del tipo: {}".format(ann[lat_idx]))
print("El latido pertence al paciente {}".format(pat[lat_idx]))
for i in range(12):
    axarr[i].plot(lat[lat_idx].T[i])
plot.show()


# Analizando datos

In [ ]:
print("Latido nro: {}".format(lat_idx))
print("Lat_shape:{}".format(np.shape(lat[lat_idx].T)))
#print("\n{}".format(lat[lat_idx].T[11]))

In [ ]:
print("Len de lat:{}".format(len(lat)))
print("Len de pat:{}".format(len(pat)))
print("Canales:12")
#Array -> 240(Latidos)x12(Canales)
#Pacientes 1-64 Train / 65-75 Test
print("Len de latidos:{}".format(len(lat[lat_idx].T[0])))

lat_idx=randint(0,len(lat))
print("\n<<<Random>>>\nLatido nro: {}".format(lat_idx))
print("El latido es del tipo: {}".format(ann[lat_idx]))
print("El latido pertence al paciente {}".format(pat[lat_idx]))

# Analizo el test

In [ ]:
# Cargo los datos de test
import pickle
with open("../input/latidositbadatasset/signals_test.pkl", 'rb') as f:
        lat=pickle.load(f)
with open("../input/latidositbadatasset/patients_test.pkl", 'rb') as f:
        pat=pickle.load(f)

In [ ]:
# Grafico el latido nro lat_idx
import numpy as np
from random import randint
import os
import matplotlib.pyplot as plot
lat_idx=randint(0,len(lat))
f, axarr = plot.subplots(12, sharex=True)
f.set_figheight(15)
f.set_figwidth(15)
print("Latido nro: {}".format(lat_idx))
print("El latido pertence al paciente {}".format(pat[lat_idx]))
for i in range(12):
    axarr[i].plot(lat[lat_idx].T[i])
plot.show()

## Preprocesamiento

## Transformaciones de tipos de datos

In [35]:
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

import sklearn
from sklearn.preprocessing import OneHotEncoder

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [92]:
#Me quedo con los distintos tipos de latidos, lo transformo en un Set para que sean unicos, y, luego como un np array.
print("Tipos de latidos en el dataset: {}".format(np.array(set(ann))))

# Transformo usando one-hot encoding de los labels
from sklearn.preprocessing import LabelBinarizer 
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(ann)))
y = label_binarizer.transform(ann)
# 
for tipo_lat in set(ann):
    print("Latido del tipo: {} - One-hot encoding: {}".format(tipo_lat,label_binarizer.transform(np.array([tipo_lat]))))
#Probando si esta correcto el one-hot encoding
random_tipo_latido=randint(0,len(ann))
print("\nLatido nro:{}".format(random_tipo_latido))
print("One-hot: {}".format(y[random_tipo_latido]))
print("Tipo Latido: {}".format(ann[random_tipo_latido]))

#print("AAA:{}".format(label_binarizer.transform(np.array(["A","N","R","V"]))))


Tipos de latidos en el dataset: {'V', 'A', 'N', 'R'}
Latido del tipo: V - One-hot encoding: [[0 0 0 1]]
Latido del tipo: A - One-hot encoding: [[1 0 0 0]]
Latido del tipo: N - One-hot encoding: [[0 1 0 0]]
Latido del tipo: R - One-hot encoding: [[0 0 1 0]]

Latido nro:50551
One-hot: [0 1 0 0]
Tipo Latido: N


## Modelo

In [ ]:
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [ ]:
#Input_shape

#Output
clases_latidos = 4
#Dropout
p = 0.25

# Creo el modelo
model=Sequential()

model.add(Convolution2D(filters=8, kernel_size=(3, 3), padding="same", name='Conv1',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool1'))
model.add(Dropout(p))

model.add(Flatten())

model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(Dense(clases_latidos, activation='softmax'))

model.summary()